In [ ]:
import boto3,re,pickle,json

sourceAccountId="xxxxxxxxxxxx"
targetAccountId="xxxxxxxxxxxx"
targetRegion='us-east-1'
dataSetId0="cncept-graph-dashboard-rollout-data"
dataSetId1="concept-graph-dashboard-feedback-data"
analysisId="concept-graph-dashboard-analysis"
themeId="concept-graph-dashboard-theme"
dashboardId="concept-graph-dashboard"
templateId="concept-graph-dashboard-template"

targetQuickSight = boto3.client('quicksight', region_name=targetRegion)

In [ ]:
#Create data set 0 on target account

file = open('./backup/sourceDataSetDescribeResponse0', 'rb')
sourceDataSetDescribeResponse0 = pickle.load(file)
file.close()
file = open('./backup/sourceDataSetPermsDescribeResponse0', 'rb')
sourceDataSetPermsDescribeResponse0 = pickle.load(file)
file.close()
sourceDataSet0=sourceDataSetDescribeResponse0['DataSet']

sourceDataSetPerms0 = sourceDataSetPermsDescribeResponse0['Permissions']
sourceDataSetPerms0[0]['Principal'] = re.sub(sourceAccountId,targetAccountId,sourceDataSetPerms0[0]['Principal'])
              
targetDataSetCreateResponse0 = targetQuickSight.create_data_set(AwsAccountId=targetAccountId,
                                      DataSetId=dataSetId0,
                                      Name=sourceDataSet0['Name'],
                                      PhysicalTableMap=sourceDataSet0['PhysicalTableMap'],
                                      LogicalTableMap=sourceDataSet0['LogicalTableMap'],
                                      ImportMode=sourceDataSet0['ImportMode'],
                                      Permissions=sourceDataSetPerms0)
json.dumps(targetDataSetCreateResponse0, default=str, indent='\t')

In [ ]:
#Describe target data set 0
targetDataSetDescribeResponse0 = targetQuickSight.describe_data_set(AwsAccountId=targetAccountId,
                                                  DataSetId=dataSetId0)
json.dumps(targetDataSetDescribeResponse0, default=str, indent='\t')

In [ ]:
#Create data set 1 on target account

file = open('./backup/sourceDataSetDescribeResponse1', 'rb')
sourceDataSetDescribeResponse1 = pickle.load(file)
file.close()
file = open('./backup/sourceDataSetPermsDescribeResponse1', 'rb')
sourceDataSetPermsDescribeResponse1 = pickle.load(file)
file.close()
sourceDataSet1=sourceDataSetDescribeResponse1['DataSet']

sourceDataSetPerms1 = sourceDataSetPermsDescribeResponse1['Permissions']
sourceDataSetPerms1[0]['Principal'] = re.sub(sourceAccountId,targetAccountId,sourceDataSetPerms1[0]['Principal'])
              
targetDataSetCreateResponse1 = targetQuickSight.create_data_set(AwsAccountId=targetAccountId,
                                      DataSetId=dataSetId1,
                                      Name=sourceDataSet1['Name'],
                                      PhysicalTableMap=sourceDataSet1['PhysicalTableMap'],
                                      LogicalTableMap=sourceDataSet1['LogicalTableMap'],
                                      ImportMode=sourceDataSet1['ImportMode'],
                                      Permissions=sourceDataSetPerms1)
json.dumps(targetDataSetCreateResponse1, default=str, indent='\t')

In [ ]:
#Describe target data set 1
targetDataSetDescribeResponse1 = targetQuickSight.describe_data_set(AwsAccountId=targetAccountId,
                                                  DataSetId=dataSetId1)
json.dumps(targetDataSetDescribeResponse1, default=str, indent='\t')

In [ ]:
#Create target theme
file = open('./backup/sourceThemeDescribeResponse', 'rb')
sourceThemeDescribeResponse = pickle.load(file)
file.close()
sourceTheme = sourceThemeDescribeResponse['Theme']
              
targetThemeCreateResponse = targetQuickSight.create_theme(AwsAccountId=targetAccountId,
                                      ThemeId=themeId,
                                      Name=sourceTheme['Name'],
                                      BaseThemeId=sourceTheme['Version']['BaseThemeId'],
                                      Configuration=sourceTheme['Version']['Configuration'])
json.dumps(targetThemeCreateResponse, default=str, indent='\t')

In [ ]:
#Describe target theme
targetThemeDescribeResponse = targetQuickSight.describe_theme(AwsAccountId=targetAccountId,
                                      ThemeId=themeId)
json.dumps(targetThemeDescribeResponse, default=str, indent='\t')

In [ ]:
#Create target Analysis
file = open('./backup/sourceAnalysisPermsDescribeResponse', 'rb')
sourceAnalysisPermsDescribeResponse = pickle.load(file)
file.close()
file = open('./backup/sourceTemplateDescribeResponse', 'rb')
sourceTemplateDescribeResponse = pickle.load(file)
file.close()
sourceAnalysisPerms = sourceAnalysisPermsDescribeResponse['Permissions']
sourceAnalysisPerms[0]['Principal'] = re.sub(sourceAccountId,targetAccountId,sourceAnalysisPerms[0]['Principal'])
sourceTemplate = sourceTemplateDescribeResponse['Template']

targetDataSet0 = targetDataSetDescribeResponse0['DataSet']
targetDataSet1 = targetDataSetDescribeResponse1['DataSet']
targetTheme = targetThemeDescribeResponse['Theme']

targetAnalysisCreateResponse = targetQuickSight.create_analysis(AwsAccountId=targetAccountId,
                                          AnalysisId=analysisId,
                                          Name=sourceAnalysis['Name'],
                                          SourceEntity={
                                                'SourceTemplate': {
                                                    'DataSetReferences': [
                                                        {
                                                            'DataSetPlaceholder': 'DataSet0',
                                                            'DataSetArn': targetDataSet0['Arn']
                                                        },
                                                        {
                                                            'DataSetPlaceholder': 'DataSet1',
                                                            'DataSetArn': targetDataSet1['Arn']
                                                        }
                                                    ],
                                                    'Arn': sourceTemplate['Arn']
                                                }
                                            },
                                          ThemeArn=targetTheme['Arn'],
                                          Permissions=sourceAnalysisPerms)
json.dumps(targetAnalysisCreateResponse, default=str, indent='\t')

In [ ]:
#Describe target analysis
targetAnalysisDescribeResponse = targetQuickSight.describe_analysis(AwsAccountId=targetAccountId,
                                      AnalysisId=analysisId)
json.dumps(targetAnalysisDescribeResponse, default=str, indent='\t')

In [ ]:
#Create target dashboard
file = open('./backup/sourceDashboardDescribeResponse', 'rb')
sourceDashboardDescribeResponse = pickle.load(file)
file.close()
file = open('./backup/sourceDashboardPermsDescribeResponse', 'rb')
sourceDashboardPermsDescribeResponse = pickle.load(file)
file.close()
sourceDashboard = sourceDashboardDescribeResponse['Dashboard']
sourceDashboardPerms = sourceDashboardPermsDescribeResponse['Permissions']
sourceDashboardPerms[0]['Principal'] = re.sub(sourceAccountId,targetAccountId,sourceDashboardPerms[0]['Principal'])
sourceTemplate = sourceTemplateDescribeResponse['Template']

targetDataSet = targetDataSetDescribeResponse['DataSet']
targetTheme = targetThemeDescribeResponse['Theme']

targetDashboardCreateResponse = targetQuickSight.create_dashboard(AwsAccountId=targetAccountId,
                                          DashboardId=dashboardId,
                                          Name=sourceDashboard['Name'],
                                          SourceEntity={
                                                'SourceTemplate': {
                                                    'DataSetReferences': [
                                                        {
                                                            'DataSetPlaceholder': 'DataSet0',
                                                            'DataSetArn': targetDataSet0['Arn']
                                                        },
                                                        {
                                                            'DataSetPlaceholder': 'DataSet1',
                                                            'DataSetArn': targetDataSet1['Arn']
                                                        }
                                                    ],
                                                    'Arn': sourceTemplate['Arn']
                                                }
                                            },
                                          ThemeArn=targetTheme['Arn'],
                                          Permissions=sourceDashboardPerms)
json.dumps(targetDashboardCreateResponse, default=str, indent='\t')

In [ ]:
#Describe target dashboard
targetDashboardDescribeResponse = targetQuickSight.describe_dashboard(AwsAccountId=targetAccountId,
                                      DashboardId=dashboardId)
json.dumps(targetDashboardDescribeResponse, default=str, indent='\t')